In [1]:
import pandas as pd
import numpy as np

In [2]:
#R library(randomForest)
from sklearn.ensemble import RandomForestRegressor

In [3]:
#R df = read.csv(file="./data/proposed-assessments-as-of-april-18th.csv", header=TRUE, sep=",")
df = pd.read_csv("data/proposed-assessments-as-of-april-18th.csv",low_memory=False)

In [4]:
# orig = read.csv(file="./data/proposed-assessments-as-of-april-18th.csv", header=TRUE, sep=",")
# OMITTED IN PORT

In [5]:
#R df = df[sapply(df, is.numeric)]
# this should select only the numeric columns
df = df.select_dtypes(exclude=['object'])

In [6]:
# insert underscores in column names
df.columns = [c.replace(' ', '_') for c in df.columns]

df.columns = df.columns.str.replace('\n','')
df.head(5)

,BlockNo,SalePrice,2017_AssmtTaxable,2017_AssmtExempt,2017_AssmtTotal,2017_TaxesTotal,YearBuilt,SquareFootage,2018_AssmtTaxable,2018_AssmtExempt,2018_AssmtTotal,2018_TaxesEstimated
0,301.0,390000.0,126000,NaN,126000,9828.0,1925.0,1600.0,416500,NaN,416500,6747.30
1,301.0,1.0,114000,NaN,114000,8892.0,1929.0,1616.0,363300,NaN,363300,5885.46
2,301.0,150000.0,124700,NaN,124700,9726.6,1929.0,1600.0,364900,NaN,364900,5911.38
3,301.0,305000.0,88000,NaN,88000,6864.0,1929.0,1582.0,335000,NaN,335000,5427.00
4,301.0,1.0,122100,NaN,122100,9523.8,1929.0,1582.0,396200,NaN,396200,6418.44


In [7]:
list(df)

['BlockNo',
 'SalePrice',
 '2017_AssmtTaxable',
 '2017_AssmtExempt',
 '2017_AssmtTotal',
 '2017_TaxesTotal',
 'YearBuilt',
 'SquareFootage',
 '2018_AssmtTaxable',
 '2018_AssmtExempt',
 '2018_AssmtTotal',
 '2018_TaxesEstimated']

In [8]:
#R undesired <- c('X2018.Assmt.Taxable', 'X2018.Assmt.Exempt', 'X2018.Assmt.Total')
undesired = ['BlockNo','2017_AssmtExempt', '2018_AssmtTaxable', '2018_AssmtExempt', '2018_AssmtTotal','2018_TaxesEstimated']

In [9]:
#R idx = !(colnames(df) %in% undesired)
idx = [x for x in list(df) if x not in undesired]
idx

['SalePrice',
 '2017_AssmtTaxable',
 '2017_AssmtTotal',
 '2017_TaxesTotal',
 'YearBuilt',
 'SquareFootage']

In [10]:
#R df = df[,idx]
# creates an index on all the columns except those in undesired
df.set_index(idx)

,,,,,,BlockNo,2017_AssmtExempt,2018_AssmtTaxable,2018_AssmtExempt,2018_AssmtTotal,2018_TaxesEstimated
SalePrice,2017_AssmtTaxable,2017_AssmtTotal,2017_TaxesTotal,YearBuilt,SquareFootage,,,,,,
390000.0,126000,126000,9828.0,1925.0,1600.0,301.0,NaN,416500,NaN,416500,6747.30
1.0,114000,114000,8892.0,1929.0,1616.0,301.0,NaN,363300,NaN,363300,5885.46
150000.0,124700,124700,9726.6,1929.0,1600.0,301.0,NaN,364900,NaN,364900,5911.38
305000.0,88000,88000,6864.0,1929.0,1582.0,301.0,NaN,335000,NaN,335000,5427.00
1.0,122100,122100,9523.8,1929.0,1582.0,301.0,NaN,396200,NaN,396200,6418.44
120000.0,90000,90000,7020.0,1929.0,1600.0,301.0,NaN,390000,NaN,390000,6318.00
1.0,124400,124400,9703.2,1929.0,1582.0,301.0,NaN,403300,NaN,403300,6533.46
282000.0,95000,95000,7410.0,1929.0,1582.0,301.0,NaN,408200,NaN,408200,6612.84
1.0,112700,112700,8790.6,1929.0,1622.0,301.0,NaN,431400,NaN,431400,6988.68


In [11]:
df[idx].describe()

,SalePrice,2017_AssmtTaxable,2017_AssmtTotal,2017_TaxesTotal,YearBuilt,SquareFootage
count,4.366100e+04,43664.000000,43664.000000,43664.000000,42897.000000,42860.000000
mean,2.651356e+05,79648.921262,100180.823974,6212.615858,1940.916265,1645.740924
std,3.720826e+05,52061.028451,53218.460518,4060.760219,46.200834,843.757809
min,0.000000e+00,700.000000,1686.000000,54.600000,1700.000000,0.000000
25%,1.000000e+04,45300.000000,67300.000000,3533.400000,1902.000000,999.750000
50%,1.850000e+05,80000.000000,92600.000000,6240.000000,1925.000000,1480.000000
75%,3.858000e+05,110000.000000,125000.000000,8580.000000,1991.000000,2136.000000
max,2.500000e+07,865400.000000,865400.000000,67501.200000,2018.000000,8927.000000


In [12]:
# replace NaN's with median of the column

#R for(i in 1:ncol(df)){
#R   df[is.na(df[,i]), i] = median(df[,i], na.rm = TRUE)
#R }

df = df.fillna(df.mean())

In [13]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df[iris.feature_names], iris.target, test_size=0.5, stratify=iris.target, random_state=123456)
X_train, X_test, y_train, y_test = train_test_split(df[idx],df['2018_TaxesEstimated'], test_size=0.2)

In [14]:
# check for NaN's
# https://datascience.stackexchange.com/questions/11928/valueerror-input-contains-nan-infinity-or-a-value-too-large-for-dtypefloat32

np.where(np.isnan(X_train))


(array([], dtype=int64), array([], dtype=int64))

In [17]:
#R rf = randomForest(X2018.Taxes.Estimated ~ ., data=df)
rf = RandomForestRegressor(n_estimators=5, random_state=2)
rf.fit(X_train, y_train)


#R diff = predict(rf, df) - df$X2018.Taxes.Estimated


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
           oob_score=False, random_state=2, verbose=0, warm_start=False)

In [18]:
#R x = seq(0,5,0.1)
#R plot(seq(0,5,0.1), sapply(x, function(w) dim(df[abs(diff)>w*sd(diff),])[1]))